### Oppgave 2) Datainnsamling

Vi henter data ut gjennom en API fra OPEN-METEO.com, Koordinatene som er satt er for Trondheim, og dataen OPEN-METEO bruker for Norge kommer fra datasettet, "MET NORDIC", som er publisert av Norges Meterologiske Institutt. OPEN-METEO er open source, og tar seg ikke betalt for dataen, den er også veldig brukervennlig, og har en liste med alle "keywords" slik at man enkelt kan manøvrere gjennom de ulike dataene


In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 63.43,
	"longitude": 10.39,
	"start_date": "1950-01-01",
	"end_date": "2025-03-22",
	"hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 63.444637298583984°N 10.36649227142334°E
Elevation 14.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
                            date  temperature_2m
0      1950-01-01 00:00:00+00:00       -6.688000
1      1950-01-01 01:00:00+00:00       -8.002001
2      1950-01-01 02:00:00+00:00       -7.402000
3      1950-01-01 03:00:00+00:00       -6.952000
4      1950-01-01 04:00:00+00:00       -6.402000
...                          ...             ...
659395 2025-03-22 19:00:00+00:00        8.118500
659396 2025-03-22 20:00:00+00:00        7.668500
659397 2025-03-22 21:00:00+00:00        7.268500
659398 2025-03-22 22:00:00+00:00        6.868500
659399 2025-03-22 23:00:00+00:00        6.568500

[659400 rows x 2 columns]


Koden bruker Open-Meteo sin API , med caching (requests_cache) og feilretting (retry_requests). Data lagres som NumPy-array og Pandas DataFrame med tidsstempler. Latitude og Longitude er variabler vi setter inn, i denne koden har vi da valgt lengde og breddegradene til Trondheim. Videre har vi start og slutt dato, her har vi da valgt å hente data fra hver time fra 1950 til 2025, for å se om jeg får hentet ut all dataen, noe jeg fikk til. "temperature_2m" er keywordet som henter ut temperatur fra APIen som er den dataen vi vil se på. Koden finner også høyde over havet, "elevation" ut i fra lengde og bredde koordinatene vi velger